In [2]:
import pickle
import sqlite3
import datetime 
from sqlalchemy import create_engine
from sqlalchemy import inspect
import pyodbc
import requests
import spotipy
import pandas as pd
from spotipy.oauth2 import SpotifyClientCredentials
import spotipy.util as util
import streamlit as st
# To make things easier later, we're also importing numpy and pandas for
# working with sample data.
import numpy as np
import matplotlib.pyplot as plt
import time


In [3]:
SPOTIPY_CLIENT_ID="ba5fae975b7f45b6949824e976fbbf8a"
SPOTIPY_CLIENT_SECRET="811fe6fa3a73436292d83c891b2fa66b"
auth_manager = SpotifyClientCredentials(client_id=SPOTIPY_CLIENT_ID, client_secret=SPOTIPY_CLIENT_SECRET)
sp = spotipy.Spotify(auth_manager=auth_manager)

#establish sql db engine
engine = create_engine("sqlite:///sql_db/spotify_features.db")

In [4]:
#get list of genres available on spotify

genre_ids=[]
for i in range(0,1000,50):
    spotify_genres=(sp.categories(country='US',limit = 50,offset=i)['categories']['items'])
    for item in spotify_genres:
        genre_ids.append(item['id'])

In [6]:
'''function to get all the genres, playlist_ids, and playlist titles for spotify curated playlists 
associated with each genre'''

def get_genre_playlists(genre_id):
    try:
        playlist_dict={}
        for i in range (0,1000,50):
            try:
                genre_playlists = sp.category_playlists(genre_id,country='US',limit = 50,offset=i)
            except ReadTimeout:
                print('Spotify timed out... trying again...')
                genre_playlists = sp.category_playlists(genre_id,country='US',limit = 50,offset=i)
            
            for item in genre_playlists['playlists']['items']:
                try:
                    if item['name'] in playlist_dict:
                        playlist_dict[item['name']].update({genre_id:item['id']})
                    else:
                        playlist_dict[item['name']]= {genre_id:item['id']}
                except TypeError or spotipy.exceptions.SpotifyException:
                    print(f'{genre_id} playlists do not exists','\n')
                    continue
        for key, value in playlist_dict.items():
            if value is None:
                del d[key]
        return playlist_dict
    except spotipy.exceptions.SpotifyException:
        pass
    
    
    

    

In [8]:
# get df with all playlists and their IDs
def get_playlist_df(genre_id):   
    genre_playlists = get_genre_playlists(genre_id)
    playlist_df_list=[]
    try:
        new_df = pd.DataFrame.from_dict(genre_playlists,orient='index')
        try:
            new_df['genre']=new_df.columns.tolist()[0]
            new_df.columns.values[0] = "playlist_id"
            playlist_df_list.append(new_df)
        except IndexError:
            pass       
    except TypeError:
        pass
    try:
        playlist_df=pd.concat(playlist_df_list)
        playlist_df=playlist_df.reset_index()
        playlist_df=playlist_df.rename(columns={"index": "playlist_name"})
        if type(playlist_df) == None:
            pass
        else:
            return playlist_df
    except ValueError:
        pass

In [12]:
#convert miliseconds to minutes
def convert_to_mins(duration_ms):
    seconds = duration_ms/1000
    minutes=seconds/60
    return minutes

#get the audio features for each song of a playlist into a dataframe and then store into sql db
def get_playlist_features(genre_id):
    df=get_playlist_df(genre_id)
    list_of_playlist_ids=df['playlist_id'].tolist()
    list_of_dfs=[]
    for playlist_id in list_of_playlist_ids:                
    #             try:
        playlist_list = sp.playlist_tracks(playlist_id,additional_types=('track', ))['items']
    #             except ReadTimeout:
    #                 print('Spotify timed out... trying again...')
    #                 playlist_list = sp.playlist_tracks(playlist_id,additional_types=('track', ))['items']
    #             try:
        track_id_dict = {}
        audio_features_dict = {}
        popularity_dict = {}
        for item in playlist_list:
            try:
                track_id_dict[item['track']['name']]=item['track']['id']
                popularity_dict[item['track']['name']]=(item['track']['popularity'])
            except TypeError:
                continue

        for key in track_id_dict:
            try:
                audio_features_dict[key]=sp.audio_features(track_id_dict[key])[0]
            except AttributeError:
                pass
        try:
            audio_df = pd.DataFrame.from_dict(audio_features_dict, orient='index')
            popularity_df = pd.DataFrame.from_dict(popularity_dict, orient='index')
            popularity_df.rename(columns={0: "popularity"},inplace = True)
            audio_df = audio_df.merge(popularity_df,how='left',left_index=True,right_index=True)
            audio_df=audio_df.reset_index()
            audio_df=audio_df.rename(columns={"index": "song_name"}) 
            audio_df['playlist_id'] = playlist_id
            audio_df = audio_df.merge(df,how='outer',on='playlist_id')
        #     display(audio_df)
            list_of_dfs.append(audio_df)
            del popularity_df
        except AttributeError:
            pass
        
    final_df = pd.concat(list_of_dfs)
    try:
        final_df['duration_ms']=final_df['duration_ms'].apply(convert_to_mins)
        final_df = final_df.rename(columns={"duration_ms": "duration_mins"})
        final_df["date_pulled"]=pd.to_datetime('today').normalize()
        final_df.drop_duplicates(subset=['song_name'],inplace=True)
        final_df=final_df.reset_index(drop=True)
    except KeyError:
        pass
    final_df.to_sql('audio_features', con=engine, index=True,if_exists='append')
    pickle.dump( final_df, open( "spotify.pkl", "wb" ) )
    return final_df

In [15]:
#final function that takes in list of all genre_ids and store into sql db

def get_all_data(list_of_genre_ids):
    for genre_id in list_of_genre_ids:
        try:
            get_playlist_features(genre_id)
            time.sleep(10)
        except TypeError:
            pass
            
    return 'All stored in database'


    

In [17]:
get_all_data(genre_ids)

'All stored in database'

In [ ]:
#chronjob scheduled for Fridays at 12:00